In [1]:
%config Completer.use_jedi = False
import sys

sys.path.insert(0, './../')
import io
import requests
import pandas as pd
import dask.dataframe as dd

from core.data_acquisition import TaxiData
from copy import deepcopy

clean_df = deepcopy(TaxiData.clean_df)

128000000

### NYC Taxi Data - [Data Page](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml)

---

Yellow Taxi File

In [4]:
def read_chunk(url: str, n_kb: int=int(1e5)) -> pd.DataFrame:
    """
    Read a chunk of data from url in kb
    """
    tries = 0
    while True:
        try:
            with requests.get(url, stream=True) as response:
                chunk = next(response.iter_content(chunk_size=n_kb))
            break
        except Exception as exc:
            tries += 1
            if tries > 5: 
                raise exc
    return pd.read_csv(io.BytesIO(chunk), error_bad_lines=False, sep=',')

url = 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-08.csv'
df1 = read_chunk(url)
print(df1.shape)
df = clean_df(df1)
df.head()

(1114, 17)


pickup_datetime  \
1 2016-08-01 00:00:02 1970-01-01 00:00:00.000000001   
  2016-08-01 00:00:03 1970-01-01 00:00:00.000000002   
  2016-08-01 00:00:04 1970-01-01 00:00:00.000000001   
2 2016-08-01 00:00:00 1970-01-01 00:00:00.000000005   
  2016-08-01 00:00:00 1970-01-01 00:00:00.000000001   

                                   dropoff_datetime  tip_amount  fare_amount  \
1 2016-08-01 00:00:02 1970-01-01 00:00:00.000000003         0.3          0.5   
  2016-08-01 00:00:03 1970-01-01 00:00:00.000000000         0.3          0.5   
  2016-08-01 00:00:04 1970-01-01 00:00:00.000000001         0.3          0.5   
2 2016-08-01 00:00:00 1970-01-01 00:00:00.000000002         0.3          0.5   
  2016-08-01 00:00:00 1970-01-01 00:00:00.000000023         0.3          0.5   

                       total_amount            vendor_id  passenger_count  \
1 2016-08-01 00:00:02           NaN  2016-08-01 00:15:04              1.0   
  2016-08-01 00:00:03           NaN  2016-08-01 00:03:45              1.0   
  2016-08-01 00:00:04           NaN  2016-08-01 00:06:12              1.0   
2 2016-08-01 00:00:00           NaN  2016-08-01 00:10:45              1.0   
  2016-08-01 00:00:00           NaN  2016-08-01 16:58:59              2.0   

                       trip_distance  payment_type  tolls_amount  
1 2016-08-01 00:00:02            NaN           0.5         17.00  
  2016-08-01 00:00:03            NaN           0.5          6.95  
  2016-08-01 00:00:04            NaN           0.5          9.35  
2 2016-08-01 00:00:00            NaN           0.5         13.50  
  2016-08-01 00:00:00            NaN           0.3         58.64